# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 22 2023 2:07PM,263422,10,ALK38467,Alikai Health,Released
1,May 22 2023 2:07PM,263422,10,ALK38548,Alikai Health,Released
2,May 22 2023 2:07PM,263422,10,ALK38638,Alikai Health,Released
3,May 22 2023 2:07PM,263422,10,ALK38642,Alikai Health,Released
4,May 22 2023 2:07PM,263422,10,ALK38644,Alikai Health,Released
5,May 22 2023 2:07PM,263422,10,ALK38646,Alikai Health,Released
6,May 22 2023 2:07PM,263422,10,ALK38651,Alikai Health,Released
7,May 22 2023 2:07PM,263422,10,ALKSAMPLE355,Alikai Health,Released
8,May 22 2023 2:07PM,263422,10,ALKSAMPLE356,Alikai Health,Released
9,May 22 2023 2:07PM,263422,10,ALK38652,Alikai Health,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,263417,Released,7
27,263418,Released,1
28,263419,Released,2
29,263420,Released,5
30,263422,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
263417,NaN,NaN,7.0
263418,NaN,NaN,1.0
263419,NaN,NaN,2.0
263420,NaN,NaN,5.0
263422,NaN,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
263367,0.0,67.0,0.0
263369,0.0,1.0,0.0
263373,0.0,12.0,0.0
263374,28.0,27.0,29.0
263376,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
263367,0,67,0
263369,0,1,0
263373,0,12,0
263374,28,27,29
263376,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,263367,0,67,0
1,263369,0,1,0
2,263373,0,12,0
3,263374,28,27,29
4,263376,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,263367,,67,
1,263369,,1,
2,263373,,12,
3,263374,28,27,29
4,263376,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 22 2023 2:07PM,263422,10,Alikai Health
11,May 22 2023 1:25PM,263420,10,Hush Hush
16,May 22 2023 1:24PM,263419,10,"Digital Brands, LLC"
18,May 22 2023 1:21PM,263418,10,"Citieffe, Inc."
19,May 22 2023 1:20PM,263417,10,"Snap Medical Industries, LLC"
26,May 22 2023 12:54PM,263416,10,"Methapharm, Inc."
27,May 22 2023 12:48PM,263415,10,"Citieffe, Inc."
31,May 22 2023 12:43PM,263414,19,"AdvaGen Pharma, Ltd"
44,May 22 2023 12:43PM,263413,20,"AdvaGen Pharma, Ltd"
45,May 22 2023 12:24PM,263406,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 22 2023 2:07PM,263422,10,Alikai Health,,,11
1,May 22 2023 1:25PM,263420,10,Hush Hush,,,5
2,May 22 2023 1:24PM,263419,10,"Digital Brands, LLC",,,2
3,May 22 2023 1:21PM,263418,10,"Citieffe, Inc.",,,1
4,May 22 2023 1:20PM,263417,10,"Snap Medical Industries, LLC",,,7
5,May 22 2023 12:54PM,263416,10,"Methapharm, Inc.",,,1
6,May 22 2023 12:48PM,263415,10,"Citieffe, Inc.",,,4
7,May 22 2023 12:43PM,263414,19,"AdvaGen Pharma, Ltd",,,13
8,May 22 2023 12:43PM,263413,20,"AdvaGen Pharma, Ltd",,,1
9,May 22 2023 12:24PM,263406,15,"Brookfield Pharmaceuticals, LLC",,,28


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 22 2023 2:07PM,263422,10,Alikai Health,11,,
1,May 22 2023 1:25PM,263420,10,Hush Hush,5,,
2,May 22 2023 1:24PM,263419,10,"Digital Brands, LLC",2,,
3,May 22 2023 1:21PM,263418,10,"Citieffe, Inc.",1,,
4,May 22 2023 1:20PM,263417,10,"Snap Medical Industries, LLC",7,,
5,May 22 2023 12:54PM,263416,10,"Methapharm, Inc.",1,,
6,May 22 2023 12:48PM,263415,10,"Citieffe, Inc.",4,,
7,May 22 2023 12:43PM,263414,19,"AdvaGen Pharma, Ltd",13,,
8,May 22 2023 12:43PM,263413,20,"AdvaGen Pharma, Ltd",1,,
9,May 22 2023 12:24PM,263406,15,"Brookfield Pharmaceuticals, LLC",28,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 22 2023 2:07PM,263422,10,Alikai Health,11,,
1,May 22 2023 1:25PM,263420,10,Hush Hush,5,,
2,May 22 2023 1:24PM,263419,10,"Digital Brands, LLC",2,,
3,May 22 2023 1:21PM,263418,10,"Citieffe, Inc.",1,,
4,May 22 2023 1:20PM,263417,10,"Snap Medical Industries, LLC",7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 22 2023 2:07PM,263422,10,Alikai Health,11.0,NaN,NaN
1,May 22 2023 1:25PM,263420,10,Hush Hush,5.0,NaN,NaN
2,May 22 2023 1:24PM,263419,10,"Digital Brands, LLC",2.0,NaN,NaN
3,May 22 2023 1:21PM,263418,10,"Citieffe, Inc.",1.0,NaN,NaN
4,May 22 2023 1:20PM,263417,10,"Snap Medical Industries, LLC",7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 22 2023 2:07PM,263422,10,Alikai Health,11.0,0.0,0.0
1,May 22 2023 1:25PM,263420,10,Hush Hush,5.0,0.0,0.0
2,May 22 2023 1:24PM,263419,10,"Digital Brands, LLC",2.0,0.0,0.0
3,May 22 2023 1:21PM,263418,10,"Citieffe, Inc.",1.0,0.0,0.0
4,May 22 2023 1:20PM,263417,10,"Snap Medical Industries, LLC",7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4214422,59.0,79.0,0.0
15,1580330,93.0,27.0,28.0
19,790188,26.0,11.0,0.0
20,263413,1.0,0.0,0.0
21,263376,1.0,0.0,0.0
22,263369,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4214422,59.0,79.0,0.0
1,15,1580330,93.0,27.0,28.0
2,19,790188,26.0,11.0,0.0
3,20,263413,1.0,0.0,0.0
4,21,263376,1.0,0.0,0.0
5,22,263369,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,59.0,79.0,0.0
1,15,93.0,27.0,28.0
2,19,26.0,11.0,0.0
3,20,1.0,0.0,0.0
4,21,1.0,0.0,0.0
5,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,59.0
1,15,Released,93.0
2,19,Released,26.0
3,20,Released,1.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21,22
Status,,,,,,
Completed,0.0,28.0,0.0,0.0,0.0,0.0
Executing,79.0,27.0,11.0,0.0,0.0,1.0
Released,59.0,93.0,26.0,1.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21,22
0,Completed,0.0,28.0,0.0,0.0,0.0,0.0
1,Executing,79.0,27.0,11.0,0.0,0.0,1.0
2,Released,59.0,93.0,26.0,1.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21,22
0,Completed,0.0,28.0,0.0,0.0,0.0,0.0
1,Executing,79.0,27.0,11.0,0.0,0.0,1.0
2,Released,59.0,93.0,26.0,1.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()